In [1]:
# Importing standard libraries

import pandas as pd
import numpy as np
import datetime as dt
import seaborn as sns

from sklearn import preprocessing
import sklearn
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_classification
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import Normalizer
from matplotlib import pyplot as plt


In [2]:

# File paths
accident_df = pd.read_csv('ACCIDENT.csv')
accident_chainage_df = pd.read_csv('ACCIDENT_CHAINAGE.csv')
accident_event_df = pd.read_csv('ACCIDENT_EVENT.csv')
accident_location_df = pd.read_csv('ACCIDENT_LOCATION.csv')
atmospheric_cond_df = pd.read_csv('ATMOSPHERIC_COND.csv')
node_id_complex_int_id_df = pd.read_csv('NODE_ID_COMPLEX_INT_ID.csv')
node_df = pd.read_csv('NODE.csv')
person_df = pd.read_csv('PERSON.csv')
road_surface_cond_df = pd.read_csv('ROAD_SURFACE_COND.csv')
subdca_df = pd.read_csv('SUBDCA.csv')
vehicle_df = pd.read_csv('VEHICLE.csv')

/var/folders/f2/7t7dy3815cxcyl6h2wq6k6380000gn/T/ipykernel_10857/4212262068.py:2: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  accident_df = pd.read_csv('ACCIDENT.csv')
/var/folders/f2/7t7dy3815cxcyl6h2wq6k6380000gn/T/ipykernel_10857/4212262068.py:4: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  accident_event_df = pd.read_csv('ACCIDENT_EVENT.csv')
/var/folders/f2/7t7dy3815cxcyl6h2wq6k6380000gn/T/ipykernel_10857/4212262068.py:5: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  accident_location_df = pd.read_csv('ACCIDENT_LOCATION.csv')
/var/folders/f2/7t7dy3815cxcyl6h2wq6k6380000gn/T/ipykernel_10857/4212262068.py:8: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  node_df = pd.read_csv('NODE.csv')
/var/folders/f2/7t7dy3815cxcyl6h2wq6k6380000gn/T/ipykernel_10857/421226206

In [3]:
# Clean and preprocess individual datasets
# PERSON dataset
age_median = person_df['AGE'].median()
person_df['AGE'].fillna(age_median, inplace=True)
person_df['SEATING_POSITION'].fillna('Unknown', inplace=True)
person_df['POSTCODE'].fillna('Unknown', inplace=True)
person_df['LICENCE_STATE'].fillna('Unknown', inplace=True)
person_df['TAKEN_HOSPITAL'].fillna('Unknown', inplace=True)
person_df.drop_duplicates(inplace=True)

# ACCIDENT_LOCATION dataset
accident_location_df['ROAD_NAME'].fillna('Unknown', inplace=True)
accident_location_df['ROAD_TYPE'].fillna('Unknown', inplace=True)
accident_location_df.drop(['NEAREST_KM_POST', 'OFF_ROAD_LOCATION'], axis=1, inplace=True)
accident_location_df['DISTANCE_LOCATION'] = pd.to_numeric(accident_location_df['DISTANCE_LOCATION'], errors='coerce')

# VEHICLE dataset
vehicle_df.drop(['VEHICLE_WEIGHT', 'CUBIC_CAPACITY'], axis=1, inplace=True)
vehicle_df['VEHICLE_POWER'] = pd.to_numeric(vehicle_df['VEHICLE_POWER'], errors='coerce')

columns_to_convert = ['VEHICLE_YEAR_MANUF', 'NO_OF_WHEELS', 'NO_OF_CYLINDERS', 'SEATING_CAPACITY']

for column in columns_to_convert:
    median_value = vehicle_df[column].median()
    vehicle_df[column].fillna(median_value, inplace=True)
    vehicle_df[column] = vehicle_df[column].astype(int)

## Convert the 'CARRY_CAPACITY' column to integers and Replace NaN values with the median
vehicle_df['CARRY_CAPACITY'] = pd.to_numeric(vehicle_df['CARRY_CAPACITY'], errors='coerce')
median_value_carry = vehicle_df['CARRY_CAPACITY'].median()
vehicle_df['CARRY_CAPACITY'].fillna(median_value_carry, inplace=True)

vehicle_df['VEHICLE_MAKE'].fillna('Unknown', inplace=True)
vehicle_df['VEHICLE_BODY_STYLE'].fillna('Unknown', inplace=True)
vehicle_df.drop(['VEHICLE_MODEL'], axis=1, inplace=True)

# ACCIDENT dataset
accident_df['PAGE'].fillna('Unknown', inplace=True)
accident_df['GRID_REFERENCE_X'].fillna('Unknown', inplace=True)
accident_df['GRID_REFERENCE_Y'].fillna('Unknown', inplace=True)
accident_df.drop(['EDITION', 'DIRECTORY'], axis=1, inplace=True)
accident_df['ACCIDENTTIME'] = pd.to_datetime(accident_df['ACCIDENTTIME'].astype(str).str.replace('.', ':', regex=True), format='%H:%M:%S').dt.time
accident_df['ACCIDENTTIME'].fillna(pd.to_datetime('00:00:00').time(), inplace=True)


# ACCIDENT_EVENT dataset
accident_event_df.dropna(inplace=True)


# Clean column names

list_df = [accident_df, accident_event_df, accident_location_df, atmospheric_cond_df, person_df, road_surface_cond_df, vehicle_df, accident_chainage_df, node_id_complex_int_id_df, node_df, subdca_df] 
for data_frame in list_df:
    column_names = data_frame.columns
    map_name = {}

    for column_name in column_names:
        map_name[column_name] = column_name.replace('_',' ').lower()

    print(map_name)
    data_frame.rename(columns=map_name, inplace=True) 




{'ACCIDENT_NO': 'accident no', 'ACCIDENTDATE': 'accidentdate', 'ACCIDENTTIME': 'accidenttime', 'ACCIDENT_TYPE': 'accident type', 'Accident Type Desc': 'accident type desc', 'DAY_OF_WEEK': 'day of week', 'Day Week Description': 'day week description', 'DCA_CODE': 'dca code', 'DCA Description': 'dca description', 'PAGE': 'page', 'GRID_REFERENCE_X': 'grid reference x', 'GRID_REFERENCE_Y': 'grid reference y', 'LIGHT_CONDITION': 'light condition', 'Light Condition Desc': 'light condition desc', 'NODE_ID': 'node id', 'NO_OF_VEHICLES': 'no of vehicles', 'NO_PERSONS': 'no persons', 'NO_PERSONS_INJ_2': 'no persons inj 2', 'NO_PERSONS_INJ_3': 'no persons inj 3', 'NO_PERSONS_KILLED': 'no persons killed', 'NO_PERSONS_NOT_INJ': 'no persons not inj', 'POLICE_ATTEND': 'police attend', 'ROAD_GEOMETRY': 'road geometry', 'Road Geometry Desc': 'road geometry desc', 'SEVERITY': 'severity', 'SPEED_ZONE': 'speed zone'}
{'ACCIDENT_NO': 'accident no', 'EVENT_SEQ_NO': 'event seq no', 'EVENT_TYPE': 'event type'

In [4]:

# Merge datasets
merged_df = pd.merge(accident_df, accident_event_df, on='accident no', how='left', suffixes=('', '_y'))
merged_df = pd.merge(merged_df, atmospheric_cond_df, on='accident no', how='left', suffixes=('', '_y'))
merged_df = pd.merge(merged_df, person_df, on='accident no', how='left', suffixes=('', '_y'))
merged_df = pd.merge(merged_df, road_surface_cond_df, on='accident no', how='left', suffixes=('', '_y'))
merged_df = pd.merge(merged_df, vehicle_df, on='accident no', how='left', suffixes=('', '_y'))
merged_df = pd.merge(merged_df, accident_location_df, on='accident no', how='left', suffixes=('', '_y'))
merged_df = pd.merge(merged_df, node_df, on='accident no', how='left', suffixes=('', '_y'))
merged_df = pd.merge(merged_df, node_id_complex_int_id_df, on='accident no', how='left', suffixes=('', '_y'))

merged_df.drop(merged_df.filter(regex='_y$').columns, axis=1, inplace=True)

# Save final datasets
#accident_chainage_df.to_csv('final_accident_chainage_df.csv', index=False)
#node_id_complex_int_id_df.to_csv('final_node_id_complex_int_id_df.csv', index=False)
#node_df.to_csv('final_node_df.csv', index=False)
#subdca_df.to_csv('final_subdca_df.csv', index=False)

#accident_event_df.to_csv('final_accident_event_df.csv', index=False)
#accident_location_df.to_csv('final_accident_location_df.csv', index=False)
#accident_df.to_csv('final_accident_df.csv', index=False)
#atmospheric_cond_df.to_csv('final_atmospheric_cond_df.csv', index=False)
#person_df.to_csv('final_person_df.csv', index=False)
#road_surface_cond_df.to_csv('final_road_surface_cond_df.csv', index=False)
#vehicle_df.to_csv('final_vehicle_df.csv', index=False)

#merged_df.to_csv('finalmerged_df.csv', index=False)

In [5]:
merged_df.columns

Index(['accident no', 'accidentdate', 'accidenttime', 'accident type',
       'accident type desc', 'day of week', 'day week description', 'dca code',
       'dca description', 'page',
       ...
       'node type', 'amg x', 'amg y', 'lga name', 'region name',
       'deg urban name', 'lat', 'long', 'postcode no', 'complex int no'],
      dtype='object', length=109)

In [10]:
for columns in merged_df.columns :
    print(columns)

accident no
accidentdate
accidenttime
accident type
accident type desc
day of week
day week description
dca code
dca description
page
grid reference x
grid reference y
light condition
light condition desc
node id
no of vehicles
no persons
no persons inj 2
no persons inj 3
no persons killed
no persons not inj
police attend
road geometry
road geometry desc
severity
speed zone
event seq no
event type
event type desc
vehicle 1 id
vehicle 1 coll pt
vehicle 1 coll pt desc
vehicle 2 id
vehicle 2 coll pt
vehicle 2 coll pt desc
person id
object type
object type desc
atmosph cond
atmosph cond seq
atmosph cond desc
vehicle id
sex
age
age group
inj level
inj level desc
seating position
helmet belt worn
road user type
road user type desc
licence state
pedest movement
postcode
taken hospital
ejected code
surface cond
surface cond desc
surface cond seq
vehicle year manuf
vehicle dca code
initial direction
road surface type
road surface type desc
reg state
vehicle body style
vehicle make
vehicle power

In [6]:
critical_data = ['accident no', 'accidentdate', 'accidenttime', 'accident type desc', 'towed away flag',  # accident info
               'lat', 'long', 'road name int', 'road type int', 'lga name', 'region name', 'deg urban name',
               'road geometry desc','amg x','amg y','postcode no', # location
               'inj level desc','severity', #outcome
               'dca description','light condition desc','speed zone','surface cond desc', #cause
               'vehicle type desc', 'vehicle year manuf', 'vehicle make', 'vehicle body style', #vehicle
               'no of vehicles', 'no persons killed','no persons not inj', 'age', 'sex', 'age group', 'road user type' # Demongraphic
               ]

final_data = merged_df[critical_data]

In [7]:
final_data.columns

Index(['accident no', 'accidentdate', 'accidenttime', 'accident type desc',
       'towed away flag', 'lat', 'long', 'road name int', 'road type int',
       'lga name', 'region name', 'deg urban name', 'road geometry desc',
       'amg x', 'amg y', 'postcode no', 'inj level desc', 'severity',
       'dca description', 'light condition desc', 'speed zone',
       'surface cond desc', 'vehicle type desc', 'vehicle year manuf',
       'vehicle make', 'vehicle body style', 'no of vehicles',
       'no persons killed', 'no persons not inj', 'age', 'sex', 'age group',
       'road user type'],
      dtype='object')

In [8]:
final_data.to_csv('final.csv', index=False)

In [9]:
final_data.columns

Index(['accident no', 'accidentdate', 'accidenttime', 'accident type desc',
       'towed away flag', 'lat', 'long', 'road name int', 'road type int',
       'lga name', 'region name', 'deg urban name', 'road geometry desc',
       'amg x', 'amg y', 'postcode no', 'inj level desc', 'severity',
       'dca description', 'light condition desc', 'speed zone',
       'surface cond desc', 'vehicle type desc', 'vehicle year manuf',
       'vehicle make', 'vehicle body style', 'no of vehicles',
       'no persons killed', 'no persons not inj', 'age', 'sex', 'age group',
       'road user type'],
      dtype='object')